## Adding:
- stellar mass via abundance matching
- r$\rm_{vir}$
- redshift

In [1]:
import numpy as np
import pandas as pd
from cosmologicalTools import CosmologicalTools as cosmo 
from abundanceMatchingClass import AM

In [2]:
snapData = np.genfromtxt('spacing.txt')
snaps = snapData[:,0]
scale = snapData[:,1]
redshift = snapData[:,2]

In [3]:
df = pd.read_csv('data_dwarfGroupSubhalos-initial.csv')
df[0:10]
cosmology = cosmo( OmegaM=0.2726, OmegaL= 0.7274, OmegaR=0, h=0.704)
mass = df['Mass at z=0']
maxMass = df['Max Mass']
maxMassTrue = maxMass*1e10
maxMassSnap = df['Max Mass Snap']
maxMassRedshifts = [redshift[snaps == i][0] for i in maxMassSnap]
maxMassStellar = [AM(maxMassTrue[i], maxMassRedshifts[i]).stellarMass()/1e10 for i in range(len(maxMassTrue))]
rvirMax = [cosmology.rvir(maxMassRedshifts[i],maxMass[i]) for i in range(len(maxMassRedshifts))]
rvir = [cosmology.rvir(0,mass[i]) for i in range(len(mass))]

df.insert(5,'Redshift',maxMassRedshifts)
df.insert(13, 'AM Stellar Mass', maxMassStellar)
df.insert(6, 'Rvir at z=0',rvir)
df.insert(7, 'Rvir at Max Mass',rvirMax)
df.drop(['Flag'],axis = 1, inplace = True)
df.to_csv('data_dwarfGroupSubhalos.csv',index=False,header=True)
df[0:10]

,Group Number,Subhalo ID,Mass at z=0,Max Mass,Max Mass Snap,Redshift,Rvir at z=0,Rvir at Max Mass,Pos x,Pos y,Pos z,Vel x,Vel y,Vel z,AM Stellar Mass
0,1432,572970,35.958447,44.805914,130,0.058507,184.997131,195.890367,37443.347656,73972.609375,47892.234375,-20.334339,-196.402298,-80.945076,1.505664
1,1432,572971,18.813280,18.813280,135,0.000000,149.068605,149.068605,37348.500000,73668.953125,47949.238281,88.393410,-258.670685,-64.499962,0.393009
2,1432,572972,18.007051,25.654199,127,0.099401,146.908034,160.927745,37289.656250,73771.554688,47781.488281,30.643660,-193.713928,22.159082,0.556410
3,1432,572973,9.972580,10.587782,132,0.033724,120.642557,121.923525,37416.652344,73484.015625,47879.484375,-27.384148,-87.556305,-212.457611,0.104411
4,1432,572974,7.261332,8.497752,86,0.987850,108.535075,98.092799,37398.605469,73688.898438,47694.597656,57.535889,-169.581238,41.627995,0.023692
5,1432,572975,3.835608,4.366576,119,0.214420,87.735645,86.762233,37260.871094,73742.523438,47990.371094,-49.997246,-375.944824,-134.908752,0.009956
6,1432,572976,3.479874,3.588174,109,0.380170,84.934836,78.648211,37264.015625,73578.367188,48042.738281,19.142351,-248.239746,-28.755138,0.005489
7,1432,572977,3.239208,3.371574,134,0.009522,82.929868,83.819524,37631.332031,73945.789062,47795.324219,-126.392715,-339.016846,-45.160450,0.007931
8,1432,572978,1.444746,1.600426,132,0.033724,63.361942,64.947793,37288.738281,74044.601562,47950.300781,81.906204,-143.163620,-169.602905,0.001311
9,1432,572979,1.175501,9.978596,123,0.152750,59.152397,115.923964,37408.390625,73527.453125,47882.898438,-53.324841,-118.236038,-194.791214,0.068915


___

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from positionCorrection import positionCorrection as correct
from mpl_toolkits.mplot3d import Axes3D
import math

In [ ]:
df = pd.read_csv('data_dwarfGroupSubhalos.csv')
df[0:10]

In [ ]:
groups      = df['Group Number']
subID       = df['Subhalo ID']
mass        = df['Mass at z=0']
maxMass     = df['Max Mass']
maxMassSnap = df['Max Mass Snap']
redshift    = df['Redshift']
rvir        = df['Rvir at z=0']
rvirMaxMass = df['Rvir at Max Mass']
position    = np.column_stack([df['Pos x'],df['Pos y'],df['Pos z']])
velocity    = np.column_stack([df['Vel x'],df['Vel y'],df['Vel z']])
stellarMass = df['AM Stellar Mass']

In [ ]:
uniqueGroups = np.unique(groups)
print(len(uniqueGroups),uniqueGroups[0])

# going to ignore groups that have subhalos with 3.2e11 Msun at z=0
uniqueGroups = [i for i in uniqueGroups if all(mass[groups == i]<32)==True] 
print(len(uniqueGroups), uniqueGroups[0])

#uniqueGroups = uniqueGroups[0:50] # for testing only

in order to get a full row out of a data frame:

$\rm df.iloc[row,:]$

to get that row as a list:

$\rm df.iloc[row,:].values[0]$

or an array:

$\rm np.array(df.iloc[row,:].values[0])$

In [ ]:
primaries = []
secondaries = []
cols = ['Group Number', 'Subhalo ID', 'Mass at z=0', 'Max Mass',
       'Max Mass Snap', 'Redshift', 'Rvir at z=0', 'Rvir at Max Mass', 'Pos x',
       'Pos y', 'Pos z', 'Vel x', 'Vel y', 'Vel z', 'AM Stellar Mass']

positionInDF = np.arange(0,len(df))
numberOfSingleSubhaloGroups = 0

for i in uniqueGroups:
    groupMask = groups == i
    masses = mass[groupMask]
    if len(masses)>1:
        firstMassive = np.sort(masses)[-1]
        secondMassive = np.sort(masses)[-2]

        locationOfPrimary = firstMassive == mass[groupMask]
        locationOfSecondary = secondMassive == mass[groupMask]
        
        primaryInfo = df.iloc[positionInDF[groupMask][locationOfPrimary],:].values[0]
        primaries.append(primaryInfo)
        
        secondaryInfo = df.iloc[positionInDF[groupMask][locationOfSecondary],:].values[0]
        secondaries.append(secondaryInfo)
    else:
        numberOfSingleSubhaloGroups = numberOfSingleSubhaloGroups+1
        
        
prims = pd.DataFrame(data = primaries, columns=cols)
prims.to_csv('data_primaries.csv',index=False,header=True)

secos = pd.DataFrame(data = secondaries, columns=cols)
secos.to_csv('data_secondaries.csv',index=False,header=True)

secos[0:10]

In [ ]:
numberOfSingleSubhaloGroups

In [8]:
import numpy as np
import pandas as pd
from cosmologicalTools import CosmologicalTools as cosmo 
from densityProfiles import NFWProfile as NFW
from positionCorrection import positionCorrection as correct


In [9]:
prims = pd.read_csv('data_primaries.csv')
secos = pd.read_csv('data_secondaries.csv')
prims[0:10]

,Group Number,Subhalo ID,Mass at z=0,Max Mass,Max Mass Snap,Redshift,Rvir at z=0,Rvir at Max Mass,Pos x,Pos y,Pos z,Vel x,Vel y,Vel z,AM Stellar Mass
0,2430.0,624505.0,29.205520,29.205520,135.0,0.000000,172.604851,172.604851,64705.785156,72975.992188,46421.535156,138.376251,-63.918503,-148.820251,0.882291
1,2440.0,624890.0,30.866114,35.831346,132.0,0.033724,175.816110,183.051612,64333.394531,65861.179688,37364.414062,48.914661,-24.234140,193.518509,1.137640
2,2558.0,629559.0,26.143052,26.143052,135.0,0.000000,166.347692,166.347692,10238.671875,71195.132812,7436.938477,250.470749,156.053787,-19.889967,0.730198
3,2696.0,634748.0,24.194411,24.194411,135.0,0.000000,162.107473,162.107473,37945.261719,33817.796875,2058.367676,49.669971,7.135126,-162.576126,0.635745
4,2724.0,635739.0,30.884163,31.855851,134.0,0.009522,175.850373,177.200123,34041.316406,59321.863281,44415.085938,-46.419659,168.707565,127.359932,0.994756
5,2803.0,638447.0,29.941808,31.813733,130.0,0.058507,174.043319,174.758983,8289.634766,843.998596,39054.925781,-188.347885,189.331390,-82.439713,0.893782
6,2853.0,640216.0,31.988968,31.988968,135.0,0.000000,177.922726,177.922726,19209.791016,21585.478516,47186.003906,9.465977,296.230896,-108.809822,1.022193
7,2881.0,641150.0,30.684861,34.528743,119.0,0.214420,175.471291,172.853147,10542.205078,31492.732422,70603.507812,-51.898743,149.577103,186.980576,0.765374
8,2938.0,643039.0,17.667863,23.017406,130.0,0.058507,145.979774,156.886929,57921.210938,62101.421875,50658.710938,184.056244,21.547897,-64.300583,0.498465
9,3004.0,645220.0,25.437599,28.069128,133.0,0.023974,164.837767,169.198152,8103.905762,18298.080078,63786.660156,413.885834,123.203636,62.403313,0.779718


In [25]:
primaryPos = np.column_stack([prims['Pos x'],prims['Pos x'],prims['Pos x']])
secondaryPos = np.column_stack([secos['Pos x'],secos['Pos x'],secos['Pos x']])
correctedSeparations = np.array([np.linalg.norm(i) for i in np.array(correct(primaryPos,secondaryPos))])

#rvirMax = [cosmology.rvir(maxMassRedshifts[i],maxMass[i]) for i in range(len(maxMassRedshifts))]
massesz0 = prims['Mass at z=0']*1e10
massesMax = prims['Max Mass']*1e10
escapez0 = [NFW(massesz0[i]).escapeVelocity(correctedSeparations[i]) for i in range(len(prims))]
escapeMax = [NFW(massesMax[i]).escapeVelocity(correctedSeparations[i]) for i in range(len(prims))]

secos.insert(15,'Escape Speed z=0',escapez0)
secos.insert(16,'Escape Speed Max Mass',escapeMax)

secos[0:10]



,Group Number,Subhalo ID,Mass at z=0,Max Mass,Max Mass Snap,Redshift,Rvir at z=0,Rvir at Max Mass,Pos x,Pos y,Pos z,Vel x,Vel y,Vel z,AM Stellar Mass,Escape Speed z=0,Escape Speed Max Mass
0,2430.0,624506.0,13.942057,15.423656,131.0,0.048524,134.898454,137.656698,64913.730469,72897.617188,46550.136719,70.240196,-1.176553,-212.714310,0.227367,163.515292,163.515292
1,2440.0,624891.0,21.038684,21.038684,135.0,0.000000,154.728699,154.728699,64414.042969,66188.351562,37334.441406,79.471931,-68.301575,188.076065,0.489338,202.999248,209.676720
2,2558.0,629560.0,24.288421,24.715602,134.0,0.009522,162.317164,162.826347,10421.964844,71323.492188,7523.081543,177.162109,151.676804,-5.727583,0.644677,166.043790,166.043790
3,2696.0,634749.0,12.284474,21.573414,132.0,0.033724,129.325313,154.570095,38066.519531,33855.480469,1952.893677,-59.210819,-52.816223,-231.186432,0.468627,179.877735,179.877735
4,2724.0,635740.0,28.676808,28.676808,135.0,0.000000,171.556938,171.556938,33988.386719,59353.902344,44731.007812,-98.837021,236.054871,32.974998,0.855825,217.491010,219.057562
5,2803.0,638448.0,16.229886,24.687775,112.0,0.328830,141.906788,150.981914,8595.202148,894.001465,39024.324219,-89.301567,187.051041,-83.222046,0.337542,148.016834,149.262459
6,2853.0,640217.0,9.095655,10.080128,133.0,0.023974,116.997344,120.265571,19210.091797,21410.617188,46904.957031,-63.809284,151.038925,6.649425,0.096169,267.065711,267.065711
7,2881.0,641151.0,18.353759,29.983924,100.0,0.575980,147.844903,155.014615,10559.490234,31355.458984,70541.867188,17.066425,228.767639,111.531136,0.364152,244.133721,252.612435
8,2938.0,643040.0,9.839462,12.896666,134.0,0.009522,120.103355,131.087144,57838.101562,62079.148438,50381.531250,263.703766,-30.609827,-110.041328,0.173119,184.458260,191.376324
9,3004.0,645221.0,7.784780,23.897342,117.0,0.243540,111.082794,151.931041,8055.973633,18282.712891,63698.984375,442.592865,156.507889,103.136993,0.363578,211.363264,215.834473


In [26]:
secos.to_csv('data_secondaries.csv',index=False,header=True)
secos[0:10]

,Group Number,Subhalo ID,Mass at z=0,Max Mass,Max Mass Snap,Redshift,Rvir at z=0,Rvir at Max Mass,Pos x,Pos y,Pos z,Vel x,Vel y,Vel z,AM Stellar Mass,Escape Speed z=0,Escape Speed Max Mass
0,2430.0,624506.0,13.942057,15.423656,131.0,0.048524,134.898454,137.656698,64913.730469,72897.617188,46550.136719,70.240196,-1.176553,-212.714310,0.227367,163.515292,163.515292
1,2440.0,624891.0,21.038684,21.038684,135.0,0.000000,154.728699,154.728699,64414.042969,66188.351562,37334.441406,79.471931,-68.301575,188.076065,0.489338,202.999248,209.676720
2,2558.0,629560.0,24.288421,24.715602,134.0,0.009522,162.317164,162.826347,10421.964844,71323.492188,7523.081543,177.162109,151.676804,-5.727583,0.644677,166.043790,166.043790
3,2696.0,634749.0,12.284474,21.573414,132.0,0.033724,129.325313,154.570095,38066.519531,33855.480469,1952.893677,-59.210819,-52.816223,-231.186432,0.468627,179.877735,179.877735
4,2724.0,635740.0,28.676808,28.676808,135.0,0.000000,171.556938,171.556938,33988.386719,59353.902344,44731.007812,-98.837021,236.054871,32.974998,0.855825,217.491010,219.057562
5,2803.0,638448.0,16.229886,24.687775,112.0,0.328830,141.906788,150.981914,8595.202148,894.001465,39024.324219,-89.301567,187.051041,-83.222046,0.337542,148.016834,149.262459
6,2853.0,640217.0,9.095655,10.080128,133.0,0.023974,116.997344,120.265571,19210.091797,21410.617188,46904.957031,-63.809284,151.038925,6.649425,0.096169,267.065711,267.065711
7,2881.0,641151.0,18.353759,29.983924,100.0,0.575980,147.844903,155.014615,10559.490234,31355.458984,70541.867188,17.066425,228.767639,111.531136,0.364152,244.133721,252.612435
8,2938.0,643040.0,9.839462,12.896666,134.0,0.009522,120.103355,131.087144,57838.101562,62079.148438,50381.531250,263.703766,-30.609827,-110.041328,0.173119,184.458260,191.376324
9,3004.0,645221.0,7.784780,23.897342,117.0,0.243540,111.082794,151.931041,8055.973633,18282.712891,63698.984375,442.592865,156.507889,103.136993,0.363578,211.363264,215.834473
